## <center>Support Vector Machine Assignment(Salary)</center>

#### Problem Statement:
<code style="background:lightblue;color:black">To Prepare a classification model using SVM for salary data.</code>

In [1]:
#Importing the necessary libraries for the model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Importing the csv file and having a glance at the test data
sal_test = pd.read_csv('SalaryData_Test(1).csv')
sal_test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [3]:
#Importing the csv file and having a glance at the train data
sal_train = pd.read_csv('SalaryData_Train(1).csv')
sal_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Making the necessary EDA for the model

In [4]:
#Checking the datatypes and null values if present any
sal_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [5]:
#Checking the datatypes and null values if present any
sal_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [6]:
#Looking into rows and columns of the data
sal_test.shape

(15060, 14)

In [7]:
#Looking into rows and columns of the data
sal_train.shape

(30161, 14)

In [8]:
#Label encoding the data 
sal_test = sal_test.apply(LabelEncoder().fit_transform)
sal_test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,8,2,1,6,4,6,3,2,1,0,0,39,37,0
1,21,2,11,8,2,4,0,4,1,0,0,49,37,0
2,11,1,7,11,2,10,0,4,1,0,0,39,37,1
3,27,2,15,9,2,6,0,2,1,87,0,39,37,1
4,17,2,0,5,4,7,1,4,1,0,0,29,37,0


In [9]:
#Label encoding the data 
sal_train = sal_train.apply(LabelEncoder().fit_transform)
sal_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,22,5,9,12,4,0,1,4,1,24,0,39,37,0
1,33,4,9,12,2,3,0,4,1,0,0,12,37,0
2,21,2,11,8,0,5,1,4,1,0,0,39,37,0
3,36,2,1,6,2,5,0,2,1,0,0,39,37,0
4,11,2,9,12,2,9,5,2,0,0,0,39,4,0


In [10]:
#Assigning the dependent and independent values for train data
x_train= sal_train.drop(['education','relationship','native','maritalstatus','sex','race'],axis=1)
y_train= sal_train['Salary']


In [11]:
#Assigning the dependent and independent values for test data
x_test= sal_test.drop(['education','relationship','native','maritalstatus','sex','race'],axis=1)
y_test=  sal_test['Salary']


### Building the SVM models

#### RBF Kernel

In [12]:
#Building the SVM model with rbf kernel and checking the accuracy
Model1 =svm.SVC(kernel='rbf',C=10,gamma=0.5)
Model1.fit(x_train,y_train)
score_Model1 = Model1.score(x_test,y_test)
print('Accuracy:',score_Model1)

Accuracy: 0.8810756972111554


 #### Poly Kernel

In [13]:
#Building the SVM model with poly kernel and checking the accuracy
Model2 =svm.SVC(kernel='poly',C=10,gamma=0.5)
Model2.fit(x_train,y_train)
score_Model2 = Model2.score(x_test,y_test)
print('Accuracy:',score_Model2) 

Accuracy: 1.0


#### Sigmoid Kernel

In [14]:
#Building the SVM model with sigmoid kernel and checking the accuracy
Model3 =svm.SVC(kernel='sigmoid',C=10,gamma=0.5)
Model3.fit(x_train,y_train)
score_Model3 = Model3.score(x_test,y_test)
print('Accuracy:',score_Model3)

Accuracy: 0.7543160690571049


In [15]:
#Predicting the accuracy on the test data
clf = SVC(kernel='poly',C=10,gamma=0.5)
clf.fit(x_train , y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)

Accuracy = 100.0


In [16]:
confusion_matrix(y_test, y_pred)

array([[11360,     0],
       [    0,  3700]], dtype=int64)

In [17]:
Table={'model':['rbf','poly','sigmoid'],
  'Acc_score':[score_Model1,score_Model2,score_Model3]
  }
Table=pd.DataFrame(Table)
Table

,model,Acc_score
0,rbf,0.881076
1,poly,1.000000
2,sigmoid,0.754316


#### Insights Drawn:
>From the table above its clear that the Poly kernel gives the better accuracy compared to the other two kernels. Hence,Poly kernel would be better to classify the salary data.